# Training Demo Enformer Celltyping

This workbook steps through training Enformer Celltyping using the full EpiMap dataset
as outlined in our manuscript. Note that steps involved including downloading data, 
preprocessing and training are both memory and computationally intensive. 

These steps are to be run on the command line. We advise running steps in parallel where 
possible and the use of at least a 24GB GPU for model training.

## Step 1. Download Data

Multiple environments are used to download and preprocess data
to deal with different package dependencies
Use the `Makefile` to create the necessary conda environments:

```
make env
```

Next download the data for the analysis by first activating the
py_analysis conda environment:

```
conda activate py_analysis
```

If this is your first time using py_analysis you will need to 
install the local package:

```
pip install -e .
```

And then run:

```
python bin/download_data_parallel.py
```

The files are downloaded to the `data/` folder. There are 103 cell types that have been 
downloaded forn EpiMap and will be used for model training (see metadata folder for more details 
on these). **Note** this download will run in parallel but will take some time. It will also need 
a decent amount of disk space (188 GB).

**Note** the downloaded files are in bigWig format taken from [EpiMap](http://compbio.mit.edu/epimap/).
BigWig files for chromatin accessibility (ATAC-Seq) or histone marks, contain the
-log10 p-value of a peak for each base-pair position of the genome. This differs from BAM
files which are just the peak regions listed at a certain threshold. Note that
although the bigWig files contain a value for every base-pair int he genome, these
have been averaged at 25 base-pairs by the peak caller (e.g. MACS2).

We use bigWig files so we can predict continuous values with our model (regression problem)
rather than 0/1 peak in a region (classification problem). This approach has been proven to
give [better performance](https://www.nature.com/articles/s42256-022-00570-9).

## Step 2. Preprocess Data

### 2.1 Liftover hg38 bigwigs to hg19

The analysis is done in hg19 but certain files are in hg38. Lift these over using the
following command, again with the py_analysis environment:

```
bash bin/liftover_to_hg19.sh
```


### 2.2 Convert all bigwigs to 128 base-pair (bp) resolution files

The model's predicitons are in 128bp resolution. However, the downloaded files are all 25bp 
resolution. So the data tracks need to be adjusted to this resolution. Use the `r_bioc` 
environment to do this and run the following script. This will take some time and will
need roughly the same amount of space as currently being used in the data folder since all
bigWigs need to be converted to 128 bp. Also, this will take a long time to run, feel
free to change the `--jobs X` amount in `bin/avg_bigwig_tracks.sh` if you have free RAM
to run more in parallel and speed it up (running on a 64 CPU, 128GB RAM machine took ~1.5 
hours):

```
bash bin/avg_bigwig_tracks.sh
```

Note you might need to change permissions on all bigwigs before running this:

```
chmod 777 ./data/*
```

**Note** I would advise to now delete all the bigWig files other than the `*_128.bigWigs`
files to save on space. 


### Normalise assay signals

The differing data for the assays correspond to the -log10(adjusted p-value) from MACS2 peak calling.
This indicates the statistical significance of a genomic position i.e. is the site likely to be a real
binding site when compared against a background. However, if we were to directly predict these scores, 
a model would perform poorly since our training samples have differing read depths.

To reduce the effect of outliers, we use arcsinh-transformed signals:

sinh^−1 𝑥=ln( 𝑥+sqrt(1+𝑥^2) )

This has been used to train [Avocado](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-01977-6) 
model and for all evaluations. Other models, such as [PREDICTD](https://www.nature.com/articles/s41467-018-03635-9)
and [Segway](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3340533/), also use this transformation to
sharpen the effect of the shape of the signal while diminishing the effect of large values.

Nothing needs to be done for this step since it is taken care of in the data generator (see 
`EnformerCelltyping/utils.py` and the `generate_data()` function)


### 2.3 Create average epigenetic mark signals

Create average epigenetic mark signals from the **training cells**. The average chromatin accessibilty
(ATAC-seq) signal is used in the model to convert the local chromatin accessibilty input to the delta
chromatin accessiblity input (cell type of interest minus avg cell type chromatin acessibility). The 
average histone mark signals were sued to measure performance against in our manuscript.
Use the `py_analysis` environment and run:

```
python3 bin/calculate_avg_track.py -m epi_mark
```
replacing epi_mark with your chosen mark to average. Or run:

```
bash bin/calculate_avg_track.sh
```
to calculate an average for all marks. Again, if you have free RAM to run in parallel it might be 
worthwhile (running on a 64 CPU, 128GB RAM machine took ~6 hours).

### 2.4 Install Enformer

If you have not done so already install Enformer. All use-cases for Enformer Celltpying involve 
transfer learning from the enformer model (with frozen weights) so download this model from 
tensorflow hub:

```
mkdir data/enformer_model/ &&\
cd data/enformer_model &&\
wget -O enformer_model.tar.gz https://tfhub.dev/deepmind/enformer/1?tf-hub-format=compressed &&\
tar xvzf enformer_model.tar.gz &&\
rm enformer_model.tar.gz &&\
cd ../../
```


## Step 3. Train Enformer Celltyping model

### 3.1 Precompute training data

You now have all the data downloaded and preprocessed to train Enformer Celltyping to predict histone marks at
different genomic regions based on DNA, local chromatin accessibility and global chromatin accessibility signals.

There is a custom built, general purpose data loader `generate_data()` (in `./EnformerCelltyping/utils.py`) which
can be used to to load the input (X) and output (Y) for any genomic position. This can be used with a tensorflow
data loader to load psoitions on-the-fly while training. However, this is not every efficient since the genomic information needs to be loaded from bigWigs containing DNA, chromatin accessibility or histone mark signals for the whole genome. Moreover, this includes passing the DNA data through the pre-trained enformer model for transfer learning each time the data is loaded.

A faster approach is to precompute the data for every training position and then save them in an efficient access file
type (here compressed numpy files - `.npz`). When training Enformer Celltyping, we used the following approach to 
identify training positions:

  1. Bin genome based on predictive window
  2. Filter bins to select training set based on DNA and cell type filters. 
      * DNA filters:
          1. Leave buffer at start/end of chromosome large enough for DNA and local chromatin accessibility windows
          2. Not in blacklist regions
      * Cell type filters:
          1. Coverage for the histone mark > 12.5% of the returned window to prioritise training on regions with peaks.
  3. Down sample resulting regions to equal the lowest count of regions for any histone mark so each hist mark has equal representation. This avoids the model biasing training on one mark. 
  

This results in 67,007 training & validation positions (cell type and genomic region combinations) and 14,188 unique genomic positions which is similar to number of positions basenji & enformer trained on (14,533). The approach ensures model sees peaks for all histone marks. The validation set positons are randomly shifted by up to a quarter of the predictive window so the model's performance doesn't overfit to the initial genomic bins.

These chosen training regions can be downloaded:

```
TO DO ADD FILE TO ONLINE REPOSITORY LIKE FIGSHARE
```

Alternatively, the script below will create the file if it is not found. To save the data for the training regions 
run the following with the `EnformerCelltyping` environment:

```
python ./bin/precomute_train_data.py  -s 0 -e 1000
```

Note this is quite time intensive so should be run in parallel and on GPU(s) if possible. It is set up to run
in parallel already as you input the row numbers of the csv of the 67,007 training regions to create.

### 3.2 Train the model

Finally, we can train the model with the following:

```
python train.py
```

Also see [training_demo.ipynb](https://github.com/neurogenomics/EnformerCelltyping/blob/master/training_demo.ipynb) for an interactive training script on a small demo dataset.